In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.appName('TransformationProductSpark').getOrCreate()

In [3]:
df_product = spark.read.csv('storage/PRECOS SEMESTRAIS - AUTOMOTIVOS.csv', header=True, inferSchema=True, sep=';')

distinct_product = df_product.select('Produto').distinct().collect()

for row in distinct_product:
    product = row['Produto']
    df_product_filter = df_product.filter(col('Produto') == product)
    df_product_filter = df_product_filter.withColumn('Data da Carga ETL', current_date())
    df_product_filter = df_product_filter.withColumn('Valor de Venda', regexp_replace(col('Valor de Venda'), ',', '.'))
    
    #Exibe registros que foram segregados por Produto 
    df_product_filter.show()
    #Salva registros segregandos por Produto
    df_product_filter.write.csv(f'storage/product/{product}/full/', header=True, mode="overwrite")

    #Agregando registros por Produto e demais
    df_product_agg = df_product_filter.groupBy(['Produto', 'Data da Coleta']).agg(round(avg('Valor de Venda'), 2).alias('Valor de Venda'))
    #Exibe registros que foram agregados
    df_product_agg.show()
    #Salva registros agregados por Produto
    df_product_agg.write.csv(f'storage/product/{product}/agg/', header=True, mode="overwrite")

spark.stop()

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------------+------------------+---------+----------+--------------+--------------+---------------+-----------------+-------------+-----------------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|            Bairro|      Cep|   Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|     Bandeira|Data da Carga ETL|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------------+------------------+---------+----------+--------------+--------------+---------------+-----------------+-------------+-----------------+
|            SE|            SP| SOROCABA|COMPETRO COMERCIO...| 00.003.188/0001-21|RUA HUMBERTO DE C...|       306|                null|    JARDIM ZULMIRA|18061-000|DIESEL S10|    03/01